In [1]:
from azureml.core import Workspace, Experiment, Environment
from azureml.core.authentication import InteractiveLoginAuthentication

workspace = "quick-starts-ws-165288"
resource_name = "aml-quickstarts-165288"
# subscription_id = "9a7511b8-150f-4a58-8528-3e7d50216c31"
subscription_id = "3e42d11f-d64d-4173-af9b-12ecaa1030b3"
ws = Workspace.get(subscription_id=subscription_id,
                   resource_group=resource_name,
                   name=workspace)
experiment = Experiment(workspace=ws, name="udacity-project-automl")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()
Environment.list(workspace=ws)

Workspace name: quick-starts-ws-165288
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-165288


{'AzureML-Triton': {
     "databricks": {
         "eggLibraries": [],
         "jarLibraries": [],
         "mavenLibraries": [],
         "pypiLibraries": [],
         "rcranLibraries": []
     },
     "docker": {
         "arguments": [],
         "baseDockerfile": null,
         "baseImage": "mcr.microsoft.com/azureml/aml-triton:20210811.v1",
         "baseImageRegistry": {
             "address": null,
             "password": null,
             "registryIdentity": null,
             "username": null
         },
         "enabled": false,
         "platform": {
             "architecture": "amd64",
             "os": "Linux"
         },
         "sharedVolumes": true,
         "shmSize": null
     },
     "environmentVariables": {
         "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
     },
     "inferencingStackVersion": null,
     "name": "AzureML-Triton",
     "python": {
         "baseCondaEnvironment": null,
         "condaDependencies": {
             "channels": [
                 "

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpuclusterautoml"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [13]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
df = pd.concat([x, y], axis=1)
df

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,1,0,0,1,7,1,116,1,999,...,0,1,0,0,0,0,0,0,0,0
32946,37,1,0,0,1,7,5,69,7,999,...,0,0,0,0,0,0,0,1,0,0
32947,26,0,0,0,0,5,2,135,4,999,...,0,0,0,0,0,0,0,1,0,0
32948,31,0,0,0,0,4,1,386,1,999,...,0,0,0,1,0,0,0,0,0,0


In [16]:
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Datastore, Dataset

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

datastore = Datastore.get(ws, 'workspaceartifactstore')
dataset = Dataset.Tabular.register_pandas_dataframe(df, datastore, "dataset_from_pandas_df", show_progress=True)

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=dataset,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=cpu_cluster_name)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/e362117f-2dfe-4d5a-9503-a9644d8bd183/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [ ]:
autoMLRun = experiment.submit(automl_config, show_output=True)
RunDetails(autoMLRun).show() 

Submitting remote run.
No run_configuration provided, running on cpuclusterautoml with default configuration
Running on remote compute: cpuclusterautoml


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-automl,AutoML_edd76516-1a19-4aee-9a1c-b7538789bb73,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
# Retrieve and save your best automl model.

best_automl_run, fitted_automl_model = autoMLRun.get_output()
print(best_automl_run)
best_automl_run_metrics = best_automl_run.get_metrics()
print(best_automl_run_metrics['accuracy'])